In [2]:
import pandas as pd

In [3]:
hash_files = [
    "results_GA_MEM_a100_b100.csv",
    "results_GA_MEM_a100_b500.csv"
]

noHash_files = [
    "results_GA_MEM_noHash_a100_b100.csv",
    "results_GA_MEM_noHash_a100_b500.csv"
]

df_hash = pd.concat([pd.read_csv(f) for f in hash_files], ignore_index=True)
df_noHash = pd.concat([pd.read_csv(f) for f in noHash_files], ignore_index=True)

print(df_hash.columns.tolist())
print(df_hash.columns.tolist())

['a', 'b', 'N', 'p', 'M', 'v', 'mean_time', 'se_time', 'mean_chi', 'se_chi', 'peak_mem_mb', 'mean_mem_mb', 'instancia']
['a', 'b', 'N', 'p', 'M', 'v', 'mean_time', 'se_time', 'mean_chi', 'se_chi', 'peak_mem_mb', 'mean_mem_mb', 'instancia']


In [6]:
# df para comparação de resultados e obtenção de estatísticas sobre eles
df_comp_MEM = pd.merge(df_hash, df_noHash, on="instancia", suffixes=('_hash', '_noHash'))

# analisar se há alguma diferença de resultado em termos de qualidade das colorações obtidas
df_comp_MEM['diff_chi'] = df_hash['mean_chi'] - df_noHash['mean_chi']

# como o que queremos verificar é se o uso da tabela Hash pode gerar redução no tempo de
# execução em troca de uso razoável de memória extra, as métricas mais importantes serão o tempo
# (com hash deve ser menor), o pico de uso de memória e a memória média (devem ser razoáveis)
# (partimos da premissa de que os resultados em qualidade de colorações serão similares ou idênticos,
# uma vez que o método é praticamente o mesmo, com alteração somente na avaliação de fitness)
df_comp_MEM['diff_tempo'] = df_hash['mean_time'] - df_noHash['mean_time']

# Percentual de alteração no tempo de execução
# se positivo: o uso de hash demorou mais (pior performance)
# se negativo: o uso de hash foi mais rápido (melhor performance) - é o que esperamos
df_comp_MEM['time_change_pct'] = ((df_comp_MEM['mean_time_hash'] - df_comp_MEM['mean_time_noHash']) / df_comp_MEM['mean_time_noHash']) * 100

df_comp_MEM['speedup'] = df_comp_MEM['mean_time_noHash'] / df_comp_MEM['mean_time_hash']

# diferenças em uso de memória (pico e médias)
df_comp_MEM['diff_mean_MEM'] = df_hash['mean_mem_mb'] - df_noHash['mean_mem_mb']

df_comp_MEM['diff_pico_MEM'] = df_hash['peak_mem_mb'] - df_noHash['peak_mem_mb']

# aumento percentual no uso de memória
df_comp_MEM['MEM_increase_pct'] = ((df_comp_MEM['mean_mem_mb_hash'] - df_comp_MEM['mean_mem_mb_noHash']) / df_comp_MEM['mean_mem_mb_noHash']) * 100

df_comp_MEM['Peak_MEM_increase_pct'] = ((df_comp_MEM['peak_mem_mb_hash'] - df_comp_MEM['peak_mem_mb_noHash']) / df_comp_MEM['peak_mem_mb_noHash']) * 100

In [7]:
# qualidade em cores
df_comp_MEM['diff_chi'] = df_comp_MEM['mean_chi_hash'] - df_comp_MEM['mean_chi_noHash']

# performance em tempo
df_comp_MEM['diff_tempo'] = df_comp_MEM['mean_time_hash'] - df_comp_MEM['mean_time_noHash']
df_comp_MEM['pct_change_time'] = ((df_comp_MEM['mean_time_hash'] - df_comp_MEM['mean_time_noHash']) / df_comp_MEM['mean_time_noHash']) * 100
df_comp_MEM['speedup'] = df_comp_MEM['mean_time_noHash'] / df_comp_MEM['mean_time_hash']
# slowdown (contrário do speedup, pode ser melhor para visualização)
df_comp_MEM['slowdown_tempo'] = df_comp_MEM['mean_time_hash'] / df_comp_MEM['mean_time_noHash']

# consumo de memória
df_comp_MEM['diff_mean_MEM'] = df_comp_MEM['mean_mem_mb_hash'] - df_comp_MEM['mean_mem_mb_noHash']
df_comp_MEM['pct_increase_mem'] = ((df_comp_MEM['mean_mem_mb_hash'] - df_comp_MEM['mean_mem_mb_noHash']) / df_comp_MEM['mean_mem_mb_noHash']) * 100
df_comp_MEM['pct_increase_peak_mem'] = ((df_comp_MEM['peak_mem_mb_hash'] - df_comp_MEM['peak_mem_mb_noHash']) / df_comp_MEM['peak_mem_mb_noHash']) * 100


# geração de estatísticas de interesse no df construído
stats_qualidade = df_comp_MEM[['mean_chi_hash', 'mean_chi_noHash', 'diff_chi']].describe()
stats_tempo = df_comp_MEM[['mean_time_hash', 'mean_time_noHash', 'diff_tempo', 'pct_change_time', 'speedup']].describe()
stats_memoria = df_comp_MEM[['mean_mem_mb_hash', 'mean_mem_mb_noHash', 'diff_mean_MEM', 'pct_increase_mem']].describe()

print("--- Estatísticas de Qualidade (Cores) ---")
print(stats_qualidade)

print("\n--- Estatísticas de Performance (Tempo) ---")
print(stats_tempo)

print("\n--- Estatísticas de Consumo (Memória) ---")
print(stats_memoria)

# resumo de vitórias e eficiência comparativas
mais_rapido = (df_comp_MEM['mean_time_hash'] < df_comp_MEM['mean_time_noHash']).sum()
tempo_igual = (df_comp_MEM['mean_time_hash'] == df_comp_MEM['mean_time_noHash']).sum()
mais_lento = (df_comp_MEM['mean_time_hash'] > df_comp_MEM['mean_time_noHash']).sum()

# casos em que o impacto de memória foi baixo (abaixo de 5%)
mem_extra_minima = (df_comp_MEM['pct_increase_mem'] < 5).sum() 

print(f"\nResumo de Eficiência (Hash vs Sem Hash):")
print(f"Hash mais rápido: {mais_rapido}")
print(f"Tempo equivalente: {tempo_igual}")
print(f"Hash mais lento: {mais_lento}")
print(f"Instâncias com baixo impacto de memória (<5%): {mem_extra_minima}")

--- Estatísticas de Qualidade (Cores) ---
       mean_chi_hash  mean_chi_noHash   diff_chi
count      24.000000        24.000000  24.000000
mean      258.725000       258.733333  -0.008333
std       212.138936       212.274577   0.652698
min        36.000000        37.800000  -1.800000
25%        91.500000        91.600000  -0.400000
50%       176.600000       176.500000   0.000000
75%       520.200000       520.150000   0.300000
max       592.400000       592.600000   1.200000

--- Estatísticas de Performance (Tempo) ---
       mean_time_hash  mean_time_noHash  diff_tempo  pct_change_time  \
count       24.000000         24.000000   24.000000        24.000000   
mean       326.150129        272.591011   53.559119        -0.473783   
std        504.858195        374.292319  186.290113        26.007347   
min          1.735520          1.841257 -120.660010       -33.475077   
25%         24.453139         30.312488   -8.635924       -14.473584   
50%         62.112122         65.108455 